# Benchmark Summary Statistics
---

Description:

    This notebook performs the summary statisticks results for all benchmark functions (one at a time).
    The optimization process is repeated N=50 times (with default settings) and then we collect the results.
    
---

### Import python libraries and set up the project directory

In [1]:
import os, sys
import numpy as np

PROJECT_DIR = os.path.abspath('..')
sys.path.append(PROJECT_DIR)

### Import the custom PSO code

In [2]:
from star_pso.population.swarm import Swarm
from star_pso.population.particle import Particle
from star_pso.engines.standard_pso import StandardPSO
from star_pso.utils.auxiliary import cost_function

ModuleNotFoundError: No module named 'population'

### Load all the Benchmark problems

In [50]:
# Dimensions: 1D
from star_pso.benchmarks.equal_maxima import EqualMaxima
from star_pso.benchmarks.five_uneven_peak_trap import FiveUnevenPeakTrap
from star_pso.benchmarks.uneven_decreasing_maxima import UnevenDecreasingMaxima

# Dimensions: 2D
from star_pso.benchmarks.himmelblau import Himmelblau
from star_pso.benchmarks.six_hump_camel_back import SixHumpCamelBack

# Dimensions: N >= 2D
from star_pso.benchmarks.shubert import Shubert
from star_pso.benchmarks.vincent import Vincent
from star_pso.benchmarks.rastrigin import Rastrigin

In [51]:
# Select one test problem.
benchmark = Vincent(n_dim=3)

### Define the cost function

Note: All the optimization problems are solved as maximizations.

In [52]:
@cost_function(minimize=False)
def func_benchmark(x_array: np.ndarray, **kwargs) -> float:
    # Return the solution.
    return benchmark.func(x_array).item()
# _end_def_

### Setup the experiment

In [53]:
# Number of repeatitions.
n_repeats = 50

# Swarm population size.
POP_SIZE = 1000

# Maximum function evaluations.
MAX_F_EVALS = 400_000

# Store the results.
results = n_repeats * [0]

# Display Info.
print(f"Testing function: {benchmark.name}\n")

# Go time ...
for i in range(n_repeats):

    # Display current iteration.
    print(f"Running optimization: {i+1} out of {n_repeats} ... ")

    # Get an initial (new) sample.
    x_sample = benchmark.sample_random_positions(n_pos=POP_SIZE)

    # Initial swarm population.
    swarm_t0 = Swarm([Particle(x) for x in x_sample])
    
    # Create the StandardPSO object that will carry on the optimization.
    test_PSO = StandardPSO(initial_swarm = swarm_t0, obj_func = func_benchmark,
                           x_min = benchmark.x_min, x_max = benchmark.x_max)
    # Run the optimization.
    test_PSO.run(max_it = 2000, f_max_eval = MAX_F_EVALS, verbose = False,
                 options = {"w0": 0.70, "c1": 1.50, "c2": 1.50, "mode": "multimodal"})
    
    # Obtain the number of the global optima that were found for epsilon = 1.0e-4.
    gopt_found, gopt_total = benchmark.search_for_optima(test_PSO.swarm.best_n(POP_SIZE))

    # Update the results.
    results[i] = gopt_found

    # Blank line to declatter the sreen.
    print(" ")
# _end_for_

print("Done.")

Testing function: Vincent_3D

Running optimization: 1 out of 50 ... 
Initial f_optimal = 0.9690
StandardPSO Reached the maximum number of function evaluations.
Final f_optimal = 1.0000
run: elapsed time = 34.239 seconds.
 
Running optimization: 2 out of 50 ... 
Initial f_optimal = 0.9796
StandardPSO Reached the maximum number of function evaluations.
Final f_optimal = 1.0000
run: elapsed time = 34.018 seconds.
 
Running optimization: 3 out of 50 ... 
Initial f_optimal = 0.9697
StandardPSO Reached the maximum number of function evaluations.
Final f_optimal = 1.0000
run: elapsed time = 34.016 seconds.
 
Running optimization: 4 out of 50 ... 
Initial f_optimal = 0.9874
StandardPSO Reached the maximum number of function evaluations.
Final f_optimal = 1.0000
run: elapsed time = 33.742 seconds.
 
Running optimization: 5 out of 50 ... 
Initial f_optimal = 0.9802
StandardPSO Reached the maximum number of function evaluations.
Final f_optimal = 1.0000
run: elapsed time = 33.783 seconds.
 
Runni

In [54]:
# Compute the "Peak Ratio".
peak_ratio = sum(results) / (gopt_total * n_repeats)

print(f"PR = {peak_ratio}")

PR = 0.22574074074074074


In [55]:
# Compute the "Success Rate".
success_rate = sum([r == gopt_total for r in results]) / n_repeats

print(f"SR = {success_rate}")

SR = 0.0


### Optimization Results

Accuracy level: $\epsilon = 1.0^{-4}$

| Benchmark Function | Dimensions | No. Global opt. | MaxFEs  | Population Size | PR | SR |
|:--|:--|:--|:--|:--|:--|:--|
| Equal Maxima | 1 | 5 | 50_000| 150 | 1.0 | 1.0 |
| Five Uneven Peak Trap | 1 | 2 | 50_000 | 200 | 1.0 | 1.0 |
| Uneven Decreasing Maxima | 1 | 1 | 50_000 | 100 | 1.0 | 1.0 |
| Himmelblau | 2 | 4 | 50_000 | 100 | 1.0 | 1.0 |
| Six Hump Camel Back | 2 | 2 | 50_000 | 100 | 1.0 | 1.0 |
| Rastrigin (modified) | 2 | 6 / 12 | 200_000 | 200 / 400 | 1.0 / 1.0 | 1.0 / 1.0 |
| Shubert | 2 / 3 | 18 / 81 | 200_000 / 400_000 | 600 / 1000 | 0.862 / 0.523 | 0.120 / 0 |
| Vincent | 2 / 3 | 36 / 216 | 200_000 / 400_000 | 600 / 1000 | 0.532 / 0.226 | 0.000 / 0 |

### End of file